# Домашнее задание

Давайте рассмотрим задачу классификации на следующие два класса:
- 1 для 'American_movie_actors'
- 0 для 'American_stage_actors'

На лекциях мы обсуждали, что вместо словаря можно использовать хэширование.

Вам предлагается проверить, как поведет себя модель после использования хэширования и ответить на следующие вопросы:
1. **Какой roc_auc_score на тестовой выборке получается при использовании словаря?**
2. **Какой roc_auc_score на тестовой выборке получается при переходе со словаря на хэширование?**

Детали:
1. Разбейте выборки на обучающую и тестовую по четности `id` статьи: четные в обучение, нечетные в тест. Только по тренировочной части мы считаем градиенты!
2. Для подсчета roc_auc_score вам нужно получить предсказания и истинные ответы для примеров из тестовой выборки. Все пары (предсказание, ответ) помещаются в память, воспользуйтесь этим!
3. В качестве хэш-функции используйте `murmurhash3_32(x) % 2**20`.
4. Зафиксируйте random seed в начальном приближении весов: `np.random.seed(0); weights = np.random.random(...)`
5. Обучите 500 эпох с шагом 0.3. После каждой эпохи вызывайте `weights_broadcast.destroy()` для удаления broadcast переменной, чтобы не закончилась память. 
6. Вот так выглядит roc_auc_score на тестовой выборке от числа эпох (чем больше roc_auc_score, тем лучше):
<img src="images/test_auc.png" width="600px"></img>

Решение сохраните в файл `result.json`. Пример содержимого файла:

```json
{
    "q1": 0.123,
    "q2": 0.456
}```

In [1]:
from sklearn.utils import murmurhash3_32

In [2]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [3]:
! hadoop fs -copyFromLocal wiki /

copyFromLocal: `/wiki/wiki.jsonl': File exists
copyFromLocal: `/wiki/README.txt': File exists
copyFromLocal: `/wiki/categories.jsonl': File exists


In [4]:
wiki = se.read.json("hdfs:///wiki/wiki.jsonl")
wiki.registerTempTable("wiki")
wiki.limit(2).toPandas()

,id,text,title,url
0,1,April\n\nApril is the fourth month of the year...,April,https://simple.wikipedia.org/wiki?curid=1
1,2,August\n\nAugust (Aug.) is the eighth month of...,August,https://simple.wikipedia.org/wiki?curid=2


In [5]:
categories = se.read.json("hdfs:///wiki/categories.jsonl")
categories.registerTempTable("categories")
categories.limit(2).toPandas()

,category,page_id
0,Months,1
1,Months,2


In [6]:
train_joined = se.sql("""
select
    wiki.text,
    cast(categories.category == 'American_movie_actors' as int) as target
from
    wiki join categories on wiki.id == categories.page_id
where categories.category in ('American_movie_actors', 'American_stage_actors')
and wiki.id % 2 = 0
""")
train_joined.limit(2).toPandas()

,text,target
0,"50 Cent\n\n50 Cent (also known as Fitty"" or ""F...",1
1,"Anne Ramsey\n\nAnne Ramsey (March 27, 1929 – A...",1


In [7]:
test_joined = se.sql("""
select
    wiki.text,
    cast(categories.category == 'American_movie_actors' as int) as target
from
    wiki join categories on wiki.id == categories.page_id
where categories.category in ('American_movie_actors', 'American_stage_actors')
and wiki.id % 2 = 1
""")
test_joined.limit(2).toPandas()

,text,target
0,"Danny Jacobs (actor)\n\nDaniel Charles Jacobs,...",1
1,Conrad Bain\n\nConrad Stafford Bain (February ...,1


In [8]:
import re
import string
import numpy as np
from collections import Counter

def tokenize(text):
    text = re.sub(f'[^{re.escape(string.printable)}]', ' ', text)  # непечатные символы заменяем на пробел
    text = re.sub(f'[{re.escape(string.punctuation)}]', ' ', text)  # и пунктуацию
    words = text.lower().split()
    return words

In [9]:
import json

def mapper(line):
    text = json.loads(line)['text']
    words = tokenize(text)
    return [(word, 1) for word in set(words)]

In [10]:
%%time
word_counts = (
    sc.textFile("hdfs:///wiki/wiki.jsonl")
    .flatMap(mapper)
    .reduceByKey(lambda a, b: a + b)
    .collect()
)

CPU times: user 82.7 ms, sys: 28.4 ms, total: 111 ms
Wall time: 18 s


In [11]:
top_word_counts = sorted(word_counts, key=lambda x: -x[1])[:50000]

In [12]:
# индексы нужны для векторизации текстов
word_to_index = {word: index for index, (word, count) in enumerate(top_word_counts)}

In [13]:
# вторая опция: broadcast переменная
word_to_index_broadcast = sc.broadcast(word_to_index)

In [14]:
def mapper(row):
    words = tokenize(row.text)
    indices = []
    values = []
    for word, count in Counter(words).items():
        if word in word_to_index:
            index = word_to_index[word]
            indices.append(index)
            tf = count / float(len(words))
            values.append(tf)
    return np.array(indices), np.array(values), row.target

In [15]:
se.udf.register("tokenize", tokenize, "array<string>")

<function __main__.tokenize(text)>

In [16]:
se.sql('''select * from wiki limit 3''').show(3)

+---+--------------------+------+--------------------+
| id|                text| title|                 url|
+---+--------------------+------+--------------------+
|  1|April

April is t...| April|https://simple.wi...|
|  2|August

August (A...|August|https://simple.wi...|
|  6|Art

Art is a cre...|   Art|https://simple.wi...|
+---+--------------------+------+--------------------+



In [17]:
train = train_joined.rdd.map(mapper)
test = test_joined.rdd.map(mapper)
train.cache()  # кэшируем датасет в RAM
train.count() + test.count()

5599

In [18]:
train.take(1)

[(array([  531,  7751,    20,    33,    11,    25,  3244,     4,    13,
            35,  4850,   733,  2384,  2618,  2669,   287,  4920,     5,
          1190,   737,    18,     6,    22,     1,  4114,    53,   225,
            62,     9,   115,   164,   778,    28,   388,    78,    42,
          1171,  6981,  1766,  1438,    75,   517,  2089,   338,   481,
          3066,    51,   313,   194,    72,   187,   802,   413,     2,
          3177,    45, 45629,  1739,   324,     7,   365,  1677,   208,
           397,  1272,   886,   226,   106,  2135,  9907,    30,    73,
           704,   236,   445,   648,    12,  1928,  4292,  5057,   201,
           303,     0,   129,   271,    48,   190,   954,  1321,   100,
            94,    23,   613,  2840,     3,   712,   381,   270,  1093,
          1275,  1468,   317,  2899,  3988,     8,   182,    27,   440,
           468,  1493,   793, 23164,   188,  1530,   150,   459,   133,
           134,   335,   963,  1805,    10,  1169,  9656,   102,

In [19]:
def sigmoid(x):
    if x >= 0:
        return 1. / (1. + np.exp(-x))
    else:
        return np.exp(x) / (1. + np.exp(x))

In [20]:
def compute_gradient(weights_broadcast, loss, examples):
    # здесь накапливаем вклад в градиент
    gradient = np.zeros(len(weights_broadcast.value))
    
    for example in examples:
        indices, values, target = example

        # делаем предсказание с текущими весами
        p = sigmoid(values.dot(weights_broadcast.value[indices]))

        # добавляем в накопитель градиента
        gradient[indices] += values * (p - target)

        # считаем потери
        p = np.clip(p, 1e-15, 1-1e-15)
        loss.add(-(target * np.log(p) + (1 - target) * np.log(1 - p)))
    
    yield gradient

In [21]:
# количество примеров
N = train.count() + test.count()

In [22]:
from functools import partial
np.random.seed(0)

# случайные веса
weights = np.random.random(len(word_to_index))

# эпохи градиентного спуска
for i in range(500):
    weights_broadcast = sc.broadcast(weights)
    loss = sc.accumulator(0.0)
    
    # считаем градиент
    gradient = (
        train
        .coalesce(2)  # склеиваем 200 кэшированных партиций в 2
        .mapPartitions(partial(compute_gradient, weights_broadcast, loss))
        .reduce(lambda a, b: a + b)
    )

    # обновляем веса
    weights -= 0.3 * gradient
    
    weights_broadcast.destroy()
    
    print("epoch:", i, "loss:", loss.value / N)

epoch: 0 loss: 0.308791626062
epoch: 1 loss: 0.30116744249
epoch: 2 loss: 0.29717618833
epoch: 3 loss: 0.295568934516
epoch: 4 loss: 0.294422814126
epoch: 5 loss: 0.29349934133
epoch: 6 loss: 0.292700359914
epoch: 7 loss: 0.291988544384
epoch: 8 loss: 0.291342982542
epoch: 9 loss: 0.290749802393
epoch: 10 loss: 0.290199061212
epoch: 11 loss: 0.289683342704
epoch: 12 loss: 0.289197023653
epoch: 13 loss: 0.288735773888
epoch: 14 loss: 0.288296216688
epoch: 15 loss: 0.287875682339
epoch: 16 loss: 0.287472031016
epoch: 17 loss: 0.287083523065
epoch: 18 loss: 0.286708724038
epoch: 19 loss: 0.28634643452
epoch: 20 loss: 0.285995638069
epoch: 21 loss: 0.285655462309
epoch: 22 loss: 0.285325149664
epoch: 23 loss: 0.285004035174
epoch: 24 loss: 0.284691529553
epoch: 25 loss: 0.284387106123
epoch: 26 loss: 0.284090290648
epoch: 27 loss: 0.283800653351
epoch: 28 loss: 0.283517802557
epoch: 29 loss: 0.283241379602
epoch: 30 loss: 0.282971054681
epoch: 31 loss: 0.282706523444
epoch: 32 loss: 0.2824

epoch: 261 loss: 0.257778849676
epoch: 262 loss: 0.257718472895
epoch: 263 loss: 0.257658268436
epoch: 264 loss: 0.257598235221
epoch: 265 loss: 0.257538372183
epoch: 266 loss: 0.257478678265
epoch: 267 loss: 0.257419152423
epoch: 268 loss: 0.257359793621
epoch: 269 loss: 0.257300600832
epoch: 270 loss: 0.257241573043
epoch: 271 loss: 0.257182709246
epoch: 272 loss: 0.257124008447
epoch: 273 loss: 0.25706546966
epoch: 274 loss: 0.257007091908
epoch: 275 loss: 0.256948874224
epoch: 276 loss: 0.25689081565
epoch: 277 loss: 0.256832915237
epoch: 278 loss: 0.256775172047
epoch: 279 loss: 0.256717585148
epoch: 280 loss: 0.256660153618
epoch: 281 loss: 0.256602876544
epoch: 282 loss: 0.256545753022
epoch: 283 loss: 0.256488782155
epoch: 284 loss: 0.256431963057
epoch: 285 loss: 0.256375294847
epoch: 286 loss: 0.256318776654
epoch: 287 loss: 0.256262407615
epoch: 288 loss: 0.256206186875
epoch: 289 loss: 0.256150113587
epoch: 290 loss: 0.256094186911
epoch: 291 loss: 0.256038406015
epoch: 292

In [23]:
def evaluate(weights_broadcast, example):
    indices, values, _ = example
    p = sigmoid(values.dot(weights_broadcast[indices]))
    return p

In [24]:
y_true = test.map(lambda x: x[2]).collect()
y_true[:5]

[1, 1, 0, 1, 0]

In [25]:
y_score = test.map(lambda x: evaluate(weights, x)).collect()
y_score[:5]

[0.75184517807023765,
 0.63053430593689674,
 0.63053430593689674,
 0.82985870394448125,
 0.82985870394448125]

In [26]:
from sklearn.metrics import roc_auc_score

# y_true - настоящие классы
# y_score - вероятности класса 1
# https://ru.wikipedia.org/wiki/ROC-кривая

answer1 = roc_auc_score(y_true, y_score)
answer1

0.68688612967379636

In [27]:
se.udf.register("murmurhash3_32", murmurhash3_32, "int")

<function sklearn.utils.murmurhash.murmurhash3_32>

In [28]:
def mapper(row):
    words = tokenize(row.text)
    d = {}
    for word in words:
        index = murmurhash3_32(word) % (2**20)
        d[index] = d.get(index, 0) + 1
        
    indices = []
    values = []
    
    for i in d:
        indices.append(i)
        values.append(d[i] / float(len(words)))
        
    return np.array(indices), np.array(values), row.target

In [29]:
train = train_joined.rdd.map(mapper)
test = test_joined.rdd.map(mapper)
train.cache()  # кэшируем датасет в RAM
train.count() + test.count()

5599

In [30]:
train.take(1)

[(array([ 333032,   12125,  554005,  626138,  494108,  791332,  408423,
          403610,  144749,  715111,  510525,  193227, 1033652,  388113,
          628081,  885734,  315967,  288863,  868051,  716217,  922115,
          764163,  824209, 1013040,  828689,  691061,  681351,  739837,
          792415,  735689, 1021892,  687296,  622766,  707072,  790616,
         1000432,  372702,  202138,  513188,  766658,  216527,   97469,
           42853,  189343,  615763,  779831,  231830,  232512,  632395,
          131095,  563656,  992412,   72944,  451990,  354738,   58931,
          242292,  466487,  798850,  754507,  174171,  824246,   69756,
           61418,  787517,   30408,  939348,  932419,  105848,  647201,
           55006,   59416,   25166,  563900,  405050,  900432,  711577,
          290475,  293337,  523298,  343796,  322712,  595059,  276572,
          761698,  824957,  602552,  892620,  240380,  326804,  799280,
            1160,  484299,  402515,  460924,  640596,  879274,  

In [31]:
def compute_gradient(weights_broadcast, loss, examples):
    # здесь накапливаем вклад в градиент
    gradient = np.zeros(2**20)
    
    for example in examples:
        indices, values, target = example

        # делаем предсказание с текущими весами
        p = sigmoid(values.dot(weights_broadcast.value[indices]))

        # добавляем в накопитель градиента
        gradient[indices] += values * (p - target)

        # считаем потери
        p = np.clip(p, 1e-15, 1-1e-15)
        loss.add(-(target * np.log(p) + (1 - target) * np.log(1 - p)))
    
    yield gradient

In [32]:
np.random.seed(0)

# случайные веса
weights = np.random.random(2**20)

# эпохи градиентного спуска
for i in range(500):
    weights_broadcast = sc.broadcast(weights)
    loss = sc.accumulator(0.0)
    
    # считаем градиент
    gradient = (
        train
        .coalesce(2)  # склеиваем 200 кэшированных партиций в 2
        .mapPartitions(partial(compute_gradient, weights_broadcast, loss))
        .reduce(lambda a, b: a + b)
    )

    # обновляем веса
    weights -= 0.3 * gradient
    
    weights_broadcast.destroy()
    
    print("epoch:", i, "loss:", loss.value / N)

epoch: 0 loss: 0.309970402493
epoch: 1 loss: 0.301442760339
epoch: 2 loss: 0.296940645776
epoch: 3 loss: 0.29527856365
epoch: 4 loss: 0.29411565607
epoch: 5 loss: 0.293188234502
epoch: 6 loss: 0.292387369813
epoch: 7 loss: 0.291673929341
epoch: 8 loss: 0.291026592678
epoch: 9 loss: 0.290431437434
epoch: 10 loss: 0.2898785317
epoch: 11 loss: 0.289360472421
epoch: 12 loss: 0.288871645219
epoch: 13 loss: 0.288407724856
epoch: 14 loss: 0.287965337758
epoch: 15 loss: 0.287541816803
epoch: 16 loss: 0.287135024945
epoch: 17 loss: 0.286743225815
epoch: 18 loss: 0.286364988844
epoch: 19 loss: 0.285999119063
epoch: 20 loss: 0.285644604954
epoch: 21 loss: 0.28530057943
epoch: 22 loss: 0.284966290459
epoch: 23 loss: 0.28464107878
epoch: 24 loss: 0.28432436087
epoch: 25 loss: 0.28401561581
epoch: 26 loss: 0.283714375059
epoch: 27 loss: 0.283420214424
epoch: 28 loss: 0.283132747674
epoch: 29 loss: 0.282851621418
epoch: 30 loss: 0.28257651094
epoch: 31 loss: 0.282307116783
epoch: 32 loss: 0.282043161

epoch: 261 loss: 0.256236210109
epoch: 262 loss: 0.256171889761
epoch: 263 loss: 0.256107749651
epoch: 264 loss: 0.256043788679
epoch: 265 loss: 0.255980005755
epoch: 266 loss: 0.255916399801
epoch: 267 loss: 0.255852969749
epoch: 268 loss: 0.255789714542
epoch: 269 loss: 0.255726633133
epoch: 270 loss: 0.255663724485
epoch: 271 loss: 0.255600987571
epoch: 272 loss: 0.255538421373
epoch: 273 loss: 0.255476024884
epoch: 274 loss: 0.255413797107
epoch: 275 loss: 0.255351737053
epoch: 276 loss: 0.255289843743
epoch: 277 loss: 0.255228116208
epoch: 278 loss: 0.255166553487
epoch: 279 loss: 0.25510515463
epoch: 280 loss: 0.255043918692
epoch: 281 loss: 0.254982844741
epoch: 282 loss: 0.254921931852
epoch: 283 loss: 0.254861179107
epoch: 284 loss: 0.2548005856
epoch: 285 loss: 0.25474015043
epoch: 286 loss: 0.254679872706
epoch: 287 loss: 0.254619751545
epoch: 288 loss: 0.254559786072
epoch: 289 loss: 0.254499975419
epoch: 290 loss: 0.254440318728
epoch: 291 loss: 0.254380815147
epoch: 292 l

In [33]:
y_score = test.map(lambda x: evaluate(weights, x)).collect()
y_score[:5]

[0.75695942519187587,
 0.62823622497408538,
 0.62823622497408538,
 0.82968514830006856,
 0.82968514830006856]

In [34]:
answer2 = roc_auc_score(y_true, y_score)
answer2

0.68776685182786357

In [37]:
%%file result.json
{
    "q1": 0.68688612967379636,
    "q2": 0.68776685182786357
}

Overwriting result.json


In [36]:
# останавливаем Spark (и YARN приложение)
# sc.stop()